# Projekt - report to pdf

# Import knihoven

In [1]:
import psycopg2
import openpyxl as opxl

import pandas as pd
import plotly.express as px

from pprint import pprint

# Import Datasetů 
- https://policie.gov.cz/clanek/statistika-nehodovosti.aspx
- Data z let 2023 - 2025
- Převedla jsem na formát xlsx
- Importováno do PostgreSQL (PostgreSQL database_dopravní nehody ČR)
- upraveny datumy a čas
- vytvořeny náhledy pro využití v další analýze

  # Připojení k databázi PostgreSQL

In [2]:
def execute_sql(sql_query: str) -> list: 
    connection = None
    data = None
    try:
        connection =  psycopg2.connect(
            host='localhost',
            user='postgres',
            password='kjm57',
            dbname='postgres')
    
        cursor = connection.cursor()
        cursor.execute(sql_query)
        data = cursor.fetchall()
        colnames = [cell[0] for cell in cursor.description]
        df_name = pd.DataFrame(data, columns=colnames)
    except Exception as E:
        print('ERROR')
    
    finally:
        if connection is not None:
            connection.close()

        return df_name

# Úprava a čištění dat

In [3]:
df_animal_involvement = execute_sql("SELECT * FROM dopravni_nehody_cr.animal_involvement")
df_accidents_in_time = execute_sql("SELECT * FROM dopravni_nehody_cr.accidents_in_time")
df_drivers = execute_sql("SELECT * FROM dopravni_nehody_cr.drivers")
df_first_aid = execute_sql("SELECT * FROM dopravni_nehody_cr.first_aid")
df_pedestrian_involvement = execute_sql("SELECT * FROM dopravni_nehody_cr.pedestrian_involvement")
df_accidents_crash = execute_sql("SELECT * FROM dopravni_nehody_cr.accidents_crash")
df_column_names = execute_sql("SELECT * FROM dopravni_nehody_cr.column_names")
df_data_description = execute_sql("SELECT * FROM dopravni_nehody_cr.data_description")

In [4]:
# Funkce pro kontrolu prázdných hodnot
def null_values(table):
    counts = table.isna().sum()
    df_na_check = pd.DataFrame({
        'column': counts.index,
        'na_count': counts.values,
        'na%': (counts.values / len(table) * 100).round(2)
    })
    df_na_check = df_na_check.merge(df_column_names,
                                    left_on='column',
                                    right_on='code',
                                    how='left')
    df_na_check = df_na_check.drop(columns=['code','table_name'])
    df_na_check = df_na_check[['column','descr','name_column_en','na_count','na%']]

    row_count = pd.DataFrame({
        'column': ['TOTAL'],
        'descr': ['Celkový počet záznamů'],
        'name_column_en': ['Total row count'],
        'na_count': [len(table)],
        'na%': [' ']
    })
    df_na_check = pd.concat([df_na_check, row_count], ignore_index=True)
    return df_na_check

def get_table_column_names(table):
    column_names = table.columns
    df_column_info = pd.DataFrame({
        'column': column_names
    })
    df_column_info = df_column_info.merge(df_column_names,
                                          left_on='column',
                                          right_on='code',
                                          how='left')
    df_column_info = df_column_info.drop(columns=['code'])
    return df_column_info


In [5]:
null_values(df_accidents_in_time)

,column,descr,name_column_en,na_count,na%
0,p1,identifikační_číslo_nehody,ID_accident,0,0.0
1,p2a,den_měsíc_rok,day_month_year,0,0.0
2,p2b,čas,time,40148,15.1
3,d,souřadnice_x,coordinate_x,0,0.0
4,e,souřadnice_y,coordinate_y,0,0.0
5,p5a,lokalita_nehody,locality,0,0.0
6,p6,druh_nehody,accident_type,0,0.0
7,p9,charakter_nehody,accident_characteristic,0,0.0
8,p13a,usmrceno_osob,fatalities,0,0.0
9,p13b,těžce_zraněno_osob,serious_injuries,0,0.0


In [6]:
df_accidents_in_time['p2a'] = pd.to_datetime(df_accidents_in_time['p2a'], format='%d.%m.%Y')
df_accidents_in_time['Year'] = df_accidents_in_time['p2a'].dt.year
df_accidents_in_time['Month'] = df_accidents_in_time['p2a'].dt.month
df_accidents_in_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265910 entries, 0 to 265909
Data columns (total 16 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   p1      265910 non-null  object        
 1   p2a     265910 non-null  datetime64[ns]
 2   p2b     225762 non-null  object        
 3   d       265910 non-null  int64         
 4   e       265910 non-null  int64         
 5   p5a     265910 non-null  object        
 6   p6      265910 non-null  object        
 7   p9      265910 non-null  object        
 8   p13a    265910 non-null  int64         
 9   p13b    265910 non-null  int64         
 10  p13c    265910 non-null  int64         
 11  p14     265908 non-null  float64       
 12  p34     265910 non-null  int64         
 13  p36     265910 non-null  object        
 14  Year    265910 non-null  int32         
 15  Month   265910 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(2), int64(6), object(6)
memory usa

In [7]:
get_table_column_names(df_accidents_in_time)

,column,descr,name_column_en,table_name
0,p1,identifikační_číslo_nehody,ID_accident,nehody
1,p2a,den_měsíc_rok,day_month_year,nehody
2,p2b,čas,time,nehody
3,d,souřadnice_x,coordinate_x,gps
4,e,souřadnice_y,coordinate_y,gps
5,p5a,lokalita_nehody,locality,nehody
6,p6,druh_nehody,accident_type,nehody
7,p9,charakter_nehody,accident_characteristic,nehody
8,p13a,usmrceno_osob,fatalities,nehody
9,p13b,těžce_zraněno_osob,serious_injuries,nehody


In [8]:
df_accidents_in_time.groupby('Year').size().reset_index(name='Count')

,Year,Count
0,2023,94945
1,2024,92218
2,2025,78747


In [9]:
df_accidents_count = df_accidents_in_time.groupby(['Year', 'Month']).size().reset_index(name='Pocet')


In [10]:
fig = px.histogram(df_accidents_in_time, 
                   x="Month", 
                   facet_col="Year", 
                   facet_col_wrap=3) # Po 3 grafech na řádek

fig.update_layout(bargap=0.1)
fig.show()

In [11]:
null_values(df_accidents_crash)

,column,descr,name_column_en,na_count,na%
0,p1,identifikační_číslo_nehody,ID_accident,0,0.0
1,p2a,den_měsíc_rok,day_month_year,0,0.0
2,p4a,kraj,region,0,0.0
3,p4b,okres,district,0,0.0
4,d,souřadnice_x,coordinate_x,0,0.0
5,e,souřadnice_y,coordinate_y,0,0.0
6,h,město,city,58,0.01
7,p5a,lokalita_nehody,locality,0,0.0
8,p8,druh_pevné_překážky,fixed_object_type,0,0.0
9,p9,charakter_nehody,accident_characteristic,0,0.0
